- LR + BOW is the best combination
- Hyper parameter tuning with this combination

In [1]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
#data ingestion
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv")

In [3]:
df = df[df['sentiment'].isin(["happiness", "sadness"])]

In [4]:
#data preprocessing
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

def removing_numbers(text):
    return ''.join([char for char in text if not char.isdigit()])

def lower_case(text):
    return " ".join([word.lower() for word in text.split()])

def removing_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    return re.sub('\s+', ' ', text).strip()

def removing_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_small_sentences(df):
    df['content'] = df['content'].apply(lambda x: np.nan if len(str(x).split()) < 3 else x)
    return df

def normalize_text(df):
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        df = remove_small_sentences(df)
        return df.dropna(subset=['content'])
    except Exception as e:
        print(e)

In [5]:
df = normalize_text(df)

In [6]:
df['sentiment'].replace({
    'sadness':0,
    'happiness':1
}, inplace=True)

In [7]:
#apply bow
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#log experiment on dagshub mlflow
import dagshub

dagshub.init(repo_owner='iamprashantjain', repo_name='mini_project', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/iamprashantjain/mini_project.mlflow")
mlflow.set_experiment("logistic regression  + bow hyper parameter tuning")

Accessing as iamprashantjain

Initialized MLflow to track repo "iamprashantjain/mini_project"

Repository iamprashantjain/mini_project initialized!

2025/05/28 19:10:28 INFO mlflow.tracking.fluent: Experiment with name 'logistic regression  + bow hyper parameter tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/84b9393b32b64689ac92c1a1f3697404', creation_time=1748439629009, experiment_id='2', last_update_time=1748439629009, lifecycle_stage='active', name='logistic regression  + bow hyper parameter tuning', tags={}>

In [10]:
import mlflow
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Start MLflow experiment
with mlflow.start_run():
    
    # Perform Grid Search
    grid_search = GridSearchCV(
        LogisticRegression(max_iter=1000),
        param_grid,
        cv=5,
        scoring='f1',
        n_jobs=-1,
        return_train_score=False
    )
    
    # Fit to training data
    grid_search.fit(X_train, y_train)

    # Iterate over results
    for i in range(len(grid_search.cv_results_['params'])):
        params = grid_search.cv_results_['params'][i]
        mean_score = grid_search.cv_results_['mean_test_score'][i]
        std_score = grid_search.cv_results_['std_test_score'][i]

        # Train a new model with these params
        model = LogisticRegression(max_iter=1000, **params)
        model.fit(X_train, y_train)

        # Evaluate
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Log metrics
        with mlflow.start_run(nested=True):
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1", f1)

    # Log best model params and score
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)


2025/05/28 19:21:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run nebulous-ape-686 at: https://dagshub.com/iamprashantjain/mini_project.mlflow/#/experiments/2/runs/7f43b429496d4c07827ccc682ab4d40d.
2025/05/28 19:21:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/mini_project.mlflow/#/experiments/2.
2025/05/28 19:21:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run luminous-owl-715 at: https://dagshub.com/iamprashantjain/mini_project.mlflow/#/experiments/2/runs/3879298979ce4f70afa95df2eab4c10e.
2025/05/28 19:21:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/mini_project.mlflow/#/experiments/2.
2025/05/28 19:21:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run worried-shark-246 at: https://dagshub.com/iamprashantjain/mini_project.mlflow/#/experiments/2/runs/10acac56d6684805973f0d75c145b75a.
2025/05/28 19:21:18 INFO mlflow.tracking._tra

#### best combination

- LR + BOW
- C = 1
- penalty = l1
- solver = liblinear
- we can create dvc pipeline on above techniques and settings
- dvc pipeline should be tracking code change & data change